In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns

In [2]:
%pip install gdown

In [5]:
CARS_FILE_ID = '1liFEe1-yFISPSpRSvbv1wIH_avYNGmBI'
RANDOM_STATE = 42

random.seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

!gdown --id {CARS_FILE_ID}

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1liFEe1-yFISPSpRSvbv1wIH_avYNGmBI
From (redirected): https://drive.google.com/uc?id=1liFEe1-yFISPSpRSvbv1wIH_avYNGmBI&confirm=t&uuid=1bcc1dde-3644-4036-8071-a720fb80b589
To: /content/dataset.csv
100% 1.01G/1.01G [00:10<00:00, 95.2MB/s]


In [6]:
df = pd.read_csv('dataset.csv')
print(df.shape)

(604047, 24)


In [7]:
df_with_na_column = df[df['engine_displacement'].isna()]
df_with_na_column

,production_year,mileage,condition,owners_number,pts_original,horse_power,accidents_resolution,region,seller_type,brand,...,engine_displacement,engine_power,fuel_rate,steering_wheel,price,price_segment,tags,auto_class,equipment,complectation_available_options
210905,2003,175000,CONDITION_OK,0,True,NaN,NaN,Чебаркуль,PRIVATE,Hyundai,...,NaN,NaN,NaN,LEFT,380000,MEDIUM,available_for_checkup;pts_original;real_photo;...,NaN,seats-5,NaN


In [8]:
df.drop(210905, inplace=True)

In [9]:
df['fuel_rate'].fillna(df['fuel_rate'].median(), inplace=True)

<ipython-input-9-b50b155cf67a>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['fuel_rate'].fillna(df['fuel_rate'].median(), inplace=True)


In [10]:
df['pts_original'].fillna(True, inplace=True)
df['accidents_resolution'].fillna('OK', inplace=True)
df['auto_class'].fillna('NOT SPECIFIED', inplace=True)

<ipython-input-10-f268849e3c66>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['pts_original'].fillna(True, inplace=True)
<ipython-input-10-f268849e3c66>:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['pts_original'].fillna(True, inplace=True)
<ipython-input-10-f268849e3c66>:2: FutureWarning: A value 

In [11]:
df.drop('horse_power', axis=1, inplace=True)

In [12]:
def get_unique_values(series, sep=';'):
    uniq = set()
    for cell in series.dropna():
        for piece in cell.split(sep):
            s = piece.strip()
            if s:
                uniq.add(s)
    return np.array(list(uniq))

all_tags = get_unique_values(df['tags'])
all_options = get_unique_values(df['complectation_available_options'])
all_equipments = get_unique_values(df['equipment'])

In [13]:
def create_binary_features(df, column, unique_values, sep=';'):
    return (
        df[column]
        .str.get_dummies(sep=sep)
        .reindex(columns=unique_values, fill_value=0)
        .astype('int8')
    )

tags_dummies = create_binary_features(df, 'tags', all_tags)
options_dummies = create_binary_features(df, 'complectation_available_options', all_options)
equipment_dummies = create_binary_features(df, 'equipment', all_equipments)

In [14]:
tags_dummies = tags_dummies.astype('Sparse[int]')
options_dummies = options_dummies.astype('Sparse[int]')
equipment_dummies = equipment_dummies.astype('Sparse[int]')

In [15]:
full_df = pd.concat([
    df.drop(columns=['tags', 'complectation_available_options', 'equipment']),
    tags_dummies,
    options_dummies.drop(columns=['condition']),
    equipment_dummies.drop(columns=['condition'])
], axis=1)

In [16]:
duplicated_cols = full_df.columns[full_df.columns.duplicated()].unique()
for col in duplicated_cols:
    full_df[col] = full_df[col].astype("int8")

In [17]:
import numpy as np

names = full_df.columns[full_df.columns.duplicated()].unique()
new_cols = {}

for name in names:
    cols_i = [col for col in full_df.columns if col == name]
    sub_df = full_df.loc[:, cols_i].astype('int8')   # DataFrame shape (n_rows, k)

    # берём максимум по строкам → Series shape (n_rows,)
    max_series = sub_df.max(axis=1).astype('int8')

    new_cols[name] = max_series

    # удаляем *все* старые колонки с этим именем
    full_df.drop(columns=cols_i, inplace=True)

# теперь присваиваем новые колонки–Series (они одномерны!)
for name, s in new_cols.items():
    full_df[name] = s


<ipython-input-17-65bc9051fcec>:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  full_df[name] = s
<ipython-input-17-65bc9051fcec>:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  full_df[name] = s
<ipython-input-17-65bc9051fcec>:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  full_df[name] = s
<

In [18]:
full_df.sample(10)

,production_year,mileage,condition,owners_number,accidents_resolution,region,seller_type,brand,model,body_type,...,rcta,body-kit,auto-mirrors,multi-wheel,ptf,combo-interior,drl,wheel-configuration2,17-inch-wheels,wheel-memory
436947,2024,0,CONDITION_OK,0,OK,Санкт-Петербург,COMMERCIAL,Skoda,Karoq,ALLROAD_5_DOORS,...,0,0,1,1,0,1,1,1,0,0
329729,2020,40460,CONDITION_OK,1,ERROR,Москва,PRIVATE,Mercedes-Benz,E-Класс,COUPE_HARDTOP,...,0,0,1,1,0,0,0,1,0,0
343497,2000,380000,CONDITION_OK,4,OK,Москва,PRIVATE,Mercedes-Benz,M-Класс,ALLROAD_5_DOORS,...,0,0,0,0,0,0,0,0,0,0
439784,2019,170000,CONDITION_OK,1,ERROR,Москва,PRIVATE,Skoda,Octavia,LIFTBACK,...,0,0,1,1,1,0,1,1,1,0
550845,2023,107,CONDITION_OK,1,OK,Краснодар,PRIVATE,Lada (ВАЗ),Largus,WAGON_5_DOORS,...,0,0,0,1,0,0,1,0,0,0
431808,2015,110727,CONDITION_OK,1,OK,Уфа,COMMERCIAL,Renault,Sandero,HATCHBACK_5_DOORS,...,0,0,0,0,1,0,0,0,0,0
167063,2024,0,CONDITION_OK,0,OK,Химки,COMMERCIAL,Genesis,GV80 Coupe,ALLROAD_5_DOORS,...,1,0,1,1,0,0,1,1,0,0
391523,2022,13000,CONDITION_OK,0,OK,Владивосток,COMMERCIAL,Nissan,X-Trail,ALLROAD_5_DOORS,...,0,0,0,0,0,0,0,0,0,0
210052,2024,100,CONDITION_OK,1,OK,Новосибирск,PRIVATE,Hyundai,Elantra,SEDAN,...,0,0,0,0,0,0,0,0,0,0
46816,2024,34,CONDITION_OK,1,OK,Москва,COMMERCIAL,BMW,X5,ALLROAD_5_DOORS,...,0,0,0,0,0,0,0,0,0,0


In [19]:
import pandas as pd
import numpy as np
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform

numeric_df = full_df.select_dtypes(include=[np.number])
numeric_df = numeric_df.loc[:, numeric_df.nunique() > 1]

corr = numeric_df.corr().abs().fillna(0)

dist = 1 - corr

dist = (dist + dist.T) / 2
np.fill_diagonal(dist.values, 0)

dist = dist.clip(lower=0)

dist_vect = squareform(dist)
Z = hierarchy.linkage(dist_vect, method='average')

max_d = 0.4
clusters = hierarchy.fcluster(Z, t=max_d, criterion='distance')

cluster_df = pd.DataFrame({'feature': numeric_df.columns, 'cluster': clusters})
to_keep = []
for c in cluster_df['cluster'].unique():
    members = cluster_df.loc[cluster_df['cluster'] == c, 'feature']
    avg_corr = corr.loc[members, members].mean().sort_values(ascending=False)
    to_keep.append(avg_corr.index[0])
to_keep = list(dict.fromkeys(to_keep))

to_drop = [col for col in numeric_df.columns if col not in to_keep]
df_reduced = full_df.drop(columns=to_drop)

print(f"Удалено числовых коррелирующих признаков: {len(to_drop)}")
print(f"Осталось признаков в df_reduced: {df_reduced.shape[1]} из {full_df.shape[1]}")

Удалено числовых коррелирующих признаков: 115
Осталось признаков в df_reduced: 301 из 416


In [20]:
df_reduced.sample(10)

,mileage,condition,accidents_resolution,region,seller_type,brand,model,body_type,doors_count,seats,...,body-mouldings,third-row-seats,auto-park,door-sill-panel,rcta,body-kit,multi-wheel,ptf,combo-interior,17-inch-wheels
518746,33000,CONDITION_OK,ERROR,Аксай,PRIVATE,Lada (ВАЗ),2107,SEDAN,4,5,...,0,0,0,0,0,0,0,0,0,0
224119,23211,CONDITION_OK,OK,Москва,PRIVATE,Hyundai,Solaris,SEDAN,4,5,...,1,0,0,0,0,0,1,1,0,0
351020,6122,CONDITION_OK,OK,Уссурийск,COMMERCIAL,Mini,Countryman,ALLROAD_5_DOORS,5,5,...,0,0,0,0,0,0,0,0,0,0
424138,4500,CONDITION_OK,ERROR,Погар,PRIVATE,Renault,Kaptur,ALLROAD_5_DOORS,5,5,...,1,0,0,0,0,0,0,0,0,0
89776,245600,CONDITION_OK,OK,Тверь,PRIVATE,Chevrolet,Aveo,SEDAN,4,5,...,0,0,0,0,0,0,0,0,0,0
297634,18300,CONDITION_OK,OK,Москва,PRIVATE,Lexus,RX,ALLROAD_5_DOORS,5,5,...,1,0,0,0,0,0,1,1,0,0
449720,0,CONDITION_OK,OK,Москва,COMMERCIAL,Soueast,S07,ALLROAD_5_DOORS,5,5;7,...,0,0,0,0,1,0,1,1,0,0
49847,121000,CONDITION_OK,OK,Москва,PRIVATE,BMW,X6,ALLROAD_5_DOORS,5,5,...,0,0,1,1,0,1,1,1,0,0
48872,0,CONDITION_OK,OK,Краснодар,COMMERCIAL,BMW,X6,ALLROAD_5_DOORS,5,5,...,0,0,0,0,0,0,0,0,0,0
846,363500,CONDITION_OK,OK,Уфа,PRIVATE,Alfa Romeo,166,SEDAN,4,5,...,0,0,0,1,0,0,0,1,0,0


## Обучим LightGBM и CatBoost на df_reduced

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
X = df_reduced.drop('price', axis=1)
y = df_reduced['price']

In [ ]:
num_features = X.select_dtypes(include=['int64', 'float64', 'int8']).columns.tolist()
cat_features = X.select_dtypes(include=['object']).columns.tolist()

In [ ]:
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), num_features),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_features),
])

**LightGBM**

In [ ]:
lgbm_pipeline = Pipeline([
    ('preproc', preprocessor),
    ('model', LGBMRegressor(objective='regression', random_state=42))
])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

In [ ]:
lgbm_pipeline.fit(X_train, y_train)

/Users/ilolss/anaconda3/envs/ml-project-2-course/lib/python3.10/site-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/Users/ilolss/anaconda3/envs/ml-project-2-course/lib/python3.10/site-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.600399 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6396
[LightGBM] [Info] Number of data points in the train set: 483236, number of used features: 2692
[LightGBM] [Info] Start training from score 2807543.358738


Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['mileage', 'doors_count',
                                                   'engine_displacement',
                                                   'engine_power', 'handling',
                                                   'safe_car', 'losangeles2017',
                                                   'catalog_china_landing_china_mark',
                                                   'catalog_photo', 'muscle',
                                                   'liquid',
                                                   'withNumericEnumFeaturePairs',
                                                   'catalog_china_landing_official_adaptation',
                                                   'user_reseller_7days',
                                                   '...
                                                   'certificate_manufacturer',
                                                   'jeneva2016',
                                                   'certificate_Favorit_Warranty', ...]),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['condition',
                                                   'accidents_resolution',
                                                   'region', 'seller_type',
                                                   'brand', 'model',
                                                   'body_type', 'seats',
                                                   'steering_wheel',
                                                   'price_segment',
                                                   'auto_class'])])),
                ('model',
                 LGBMRegressor(objective='regression', random_state=42))])

In [ ]:
y_pred_lgbm = lgbm_pipeline.predict(X_test)

/Users/ilolss/anaconda3/envs/ml-project-2-course/lib/python3.10/site-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/Users/ilolss/anaconda3/envs/ml-project-2-course/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [ ]:
mse_lgbm = mean_squared_error(y_test, y_pred_lgbm)
mae_lgbm = mean_absolute_error(y_test, y_pred_lgbm)
r2_lgbm  = r2_score(y_test, y_pred_lgbm)

In [ ]:
print("LightGBM:")
print(f"  MSE:  {mse_lgbm:.2f}")
print(f"  MAE:  {mae_lgbm:.2f}")
print(f"  R^2:  {r2_lgbm:.2f}")

LightGBM:
  MSE:  7009385199619.20
  MAE:  474050.08
  R^2:  0.75


**CatBoost**

In [ ]:
cat_pipeline = Pipeline([
    ('preproc', preprocessor),
    ('model', CatBoostRegressor(loss_function='RMSE', verbose=0, random_seed=RANDOM_STATE))
])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

In [ ]:
cat_pipeline.fit(X_train, y_train)

/Users/ilolss/anaconda3/envs/ml-project-2-course/lib/python3.10/site-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/Users/ilolss/anaconda3/envs/ml-project-2-course/lib/python3.10/site-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['mileage', 'doors_count',
                                                   'engine_displacement',
                                                   'engine_power', 'handling',
                                                   'safe_car', 'losangeles2017',
                                                   'catalog_china_landing_china_mark',
                                                   'catalog_photo', 'muscle',
                                                   'liquid',
                                                   'withNumericEnumFeaturePairs',
                                                   'catalog_china_landing_official_adaptation',
                                                   'user_reseller_7days',
                                                   '...
                                                   'certificate_manufacturer',
                                                   'jeneva2016',
                                                   'certificate_Favorit_Warranty', ...]),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['condition',
                                                   'accidents_resolution',
                                                   'region', 'seller_type',
                                                   'brand', 'model',
                                                   'body_type', 'seats',
                                                   'steering_wheel',
                                                   'price_segment',
                                                   'auto_class'])])),
                ('model',
                 <catboost.core.CatBoostRegressor object at 0x161247100>)])

In [ ]:
y_pred_cat  = cat_pipeline.predict(X_test)

/Users/ilolss/anaconda3/envs/ml-project-2-course/lib/python3.10/site-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [ ]:
mse_cat = mean_squared_error(y_test, y_pred_cat)
mae_cat = mean_absolute_error(y_test, y_pred_cat)
r2_cat  = r2_score(y_test, y_pred_cat)

In [ ]:
print("LightGBM:")
print(f"  MSE:  {mse_lgbm:.2f}")
print(f"  MAE:  {mae_lgbm:.2f}")
print(f"  R^2:  {r2_lgbm:.2f}")

print("CatBoost:")
print(f"  MSE:  {mse_cat:.2f}")
print(f"  MAE:  {mae_cat:.2f}")
print(f"  R^2:  {r2_cat:.2f}")

LightGBM:
  MSE:  7009385199619.20
  MAE:  474050.08
  R^2:  0.75
CatBoost:
  MSE:  8188312842766.43
  MAE:  417499.35
  R^2:  0.71


## Попробуем подобрать гиперпараметры с помощью Bayesian Optimization

**CatBoost**

In [24]:
import optuna
from optuna.trial import TrialState
from catboost import CatBoostRegressor, utils as cb_utils
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.metrics import r2_score
import joblib

In [25]:
X = df_reduced.drop('price', axis=1)
y = df_reduced['price']

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

In [26]:
num_features = X.select_dtypes(include=['int64','float64','int8']).columns.tolist()
cat_features = X.select_dtypes(include=['object']).columns.tolist()

preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_features),
    ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), cat_features),
])

In [27]:
X_train_proc = preprocessor.fit_transform(X_train)
X_valid_proc = preprocessor.transform(X_valid)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [28]:
n_gpus = cb_utils.get_gpu_device_count()
if n_gpus == 0:
    raise RuntimeError("GPU не найдена!")
devices = ",".join(str(i) for i in range(n_gpus))

In [29]:
N_TRIALS = 50

def objective(trial):
    params = {
        'task_type':     'GPU',
        'devices':       devices,
        'gpu_ram_part':  0.8,
        'iterations':    trial.suggest_int('iterations',   200, 2000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
        'depth':         trial.suggest_int('depth',        4,  10),
        'l2_leaf_reg':   trial.suggest_loguniform('l2_leaf_reg',  1e-1, 10.0),
        'border_count':  trial.suggest_int('border_count', 32, 255),
        'random_seed':   42,
        'verbose':       False,
    }
    model = CatBoostRegressor(**params)
    model.fit(
        X_train_proc, y_train,
        eval_set=(X_valid_proc, y_valid),
        early_stopping_rounds=50
    )
    preds = model.predict(X_valid_proc)
    return r2_score(y_valid, preds)

def progress_cb(study, trial):
    done = len([t for t in study.trials if t.state == TrialState.COMPLETE])
    print(f"Completed {done}/{N_TRIALS} trials")

In [31]:
study = optuna.create_study(direction='maximize')
study.optimize(
    objective,
    n_trials=N_TRIALS,
    n_jobs=1,
    show_progress_bar=True,
    callbacks=[progress_cb]
)

best = study.best_params
best.update({'task_type':'GPU','devices':devices,'gpu_ram_part':0.8,'random_seed':RANDOM_STATE,'verbose':False})
final_model = CatBoostRegressor(**best)
final_model.fit(X_train_proc, y_train, eval_set=(X_valid_proc, y_valid), early_stopping_rounds=N_TRIALS)

joblib.dump(final_model, 'best_catboost_model.pkl')
joblib.dump(preprocessor,      'preprocessor.pkl')
print("Models saved to best_catboost_model.pkl and preprocessor.pkl")

[I 2025-06-09 12:57:30,850] A new study created in memory with name: no-name-7a57941d-220e-4e34-8401-883bd68ef5d8


  0%|          | 0/50 [00:00<?, ?it/s]

<ipython-input-29-838bf081fba2>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
<ipython-input-29-838bf081fba2>:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg':   trial.suggest_loguniform('l2_leaf_reg',  1e-1, 10.0),


[I 2025-06-09 12:59:05,155] Trial 0 finished with value: 0.6932672294177961 and parameters: {'iterations': 794, 'learning_rate': 0.06569379466901307, 'depth': 7, 'l2_leaf_reg': 1.3630312367860937, 'border_count': 94}. Best is trial 0 with value: 0.6932672294177961.
Completed 1/50 trials


<ipython-input-29-838bf081fba2>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
<ipython-input-29-838bf081fba2>:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg':   trial.suggest_loguniform('l2_leaf_reg',  1e-1, 10.0),


...

[I 2025-06-09 14:12:49,077] Trial 47 finished with value: 0.7203897307286713 and parameters: {'iterations': 759, 'learning_rate': 0.014814267610488948, 'depth': 10, 'l2_leaf_reg': 3.260095243198889, 'border_count': 241}. Best is trial 45 with value: 0.7368262842322233.
Completed 48/50 trials


<ipython-input-29-838bf081fba2>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
<ipython-input-29-838bf081fba2>:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg':   trial.suggest_loguniform('l2_leaf_reg',  1e-1, 10.0),


[I 2025-06-09 14:14:21,005] Trial 48 finished with value: 0.7286226655734462 and parameters: {'iterations': 542, 'learning_rate': 0.06191130781824827, 'depth': 9, 'l2_leaf_reg': 5.347437704500289, 'border_count': 255}. Best is trial 45 with value: 0.7368262842322233.
Completed 49/50 trials


<ipython-input-29-838bf081fba2>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
<ipython-input-29-838bf081fba2>:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg':   trial.suggest_loguniform('l2_leaf_reg',  1e-1, 10.0),


[I 2025-06-09 14:15:52,667] Trial 49 finished with value: 0.7230531860931899 and parameters: {'iterations': 538, 'learning_rate': 0.06027005210870467, 'depth': 9, 'l2_leaf_reg': 5.964187730957609, 'border_count': 255}. Best is trial 45 with value: 0.7368262842322233.
Completed 50/50 trials
Models saved to best_catboost_model.pkl and preprocessor.pkl


In [32]:
print("Best R²:", study.best_value)
print("Best params:", study.best_params)

Best R²: 0.7368262842322233
Best params: {'iterations': 599, 'learning_rate': 0.05951760389847296, 'depth': 10, 'l2_leaf_reg': 2.04641012682822, 'border_count': 253}


### Выводы

$R^2 = 0.7368$

In [ ]:
param = {
    'iterations': 599, 
    'learning_rate': 0.05951760389847296, 
    'depth': 10, 
    'l2_leaf_reg': 2.04641012682822, 
    'border_count': 253
}